# Levelling Up

When we left off at the last section, we had built up the mathematical machinery to come up with the ideal time to arrive at the airport. This machinery was built on top of a specific cost model, parameterized by a single value $R$, which we called the cost ratio, which reflects the number of hours one would need to wait at the airport before they considered their experience as bad as having missed the flight. Moreover, we actually _calculated_ that ideal time as a function of the cost ratio by adopting a model for the distribution of wait times and then _fitting_ that model to some very hand-wavy data provided by the TSA.

However, we found the values that popped out of this model a bit optimistic, both because the exponential distribution we picked as a model wasn't realistic, and because the data we were using was bad. If we can adopt a better model, and fit it with better data, maybe we could arrive at values that align better with our experience.

To do so, I'm going to continue pretending that all the time of interest is involved in security checkpoints. My reason for this is both because it tends to have the most data available, and because it captures the most variability. Compared to the variance in this distribution, everything else (e.g. the time it takes you to walk to the gate) represents basically a constant offset, which I'll let you add depending on what airport you're going to.

## The Log-Normal Distribution
One of the biggest issues we observed with the exponential distribution from the last section was that its maximum value was at $t=0$. This is problematic for us because, as we know, TSA wait times taking no time is the _least_ likely outcome of all. A model distribution which might be a step in the right direction is the [log-normal distribution](https://en.wikipedia.org/wiki/Log-normal_distribution), the distribution for variables whose logarithm is distributed like a Gaussian. Like the Gaussian, it is parameterized by two values, the mean $\mu$ and variance $\sigma^2$, in contrast to the exponential distribution from the last section, which was parameterized by only one value, $\lambda$. However, unlike the regular Gaussian, it _only_ supports positive values, as our use case obviously requires.

Let's take a look at a couple different log-normal distributions to get a sense for what they look like before we try to do any fitting.

In [1]:
import numpy as np
from scipy.stats import lognorm
from itertools import product
from bokeh.io import output_notebook, show
import plot_utils

output_notebook()

sigmas = [0.2, 0.5, 1]
mus = [1, 2]

dists = {}
for mu, sigma in product(mus, sigmas):
    dists["Mu: {}, Sigma: {:0.1f}".format(mu, sigma)] = lognorm(s=sigma, scale=np.exp(mu))
plot_kwargs = {
    "title": "Likelihood of Wait Times",
    "x_axis_label": "Time (minutes)",
    "y_axis_label": "Probability"
}
p = plot_utils.plot_distributions(dists, x_min=0, x_max=20, plot_kwargs=plot_kwargs)
show(p)

Loading BokehJS ...

Now _this_ distribution is starting to look more realistic. For one, it always has $p(0) = 0$, which is a relief. Moreover, it tends to have a chunk that looks Gaussian-ish, and falls off exponentially-ish on either side. This should at least sort of align with our intuition about wait times, in that there's a range of times we know are pretty likely, say 20-30 minutes for a typical "busy" day at the airport, but times well outside this range on either side like 5 minutes or 45 minutes are pretty anomalous.

So last time we came up with an equation for the ideal arrival time as a function of the parameters of our model, $\lambda$, and the parameters of our cost function, $R$,

$$\tau^* = \frac{1}{\lambda}\log(\lambda R + 1),$$

which was useful because then once we fit a value of $\lambda$ to some crappy data, we could compute $\tau^*$ as a function of just $R$, for which we can just plug in our own personal value to get our ideal arrival time. Can we come up with a similar equation for the log-normal distribution, starting from the equation

$$\frac{d\log{P(t)}}{dt}\biggr\rvert_{\tau^*} = \frac{1}{R}$$

and using the cumulative density function of the log-normal distribution

$$P(T \leq t) = \frac{1}{2} + \frac{1}{2}\text{erf}\big{[}\frac{\log{t} - \mu}{\sqrt{2}\sigma}\big{]}\ \ ?$$

I sure can't! (Though if you can, please let me know...) Luckily, I have a computer that can compute this function really fast, so I can just evaluate $\frac{d\log{P(t)}}{dt}\biggr\rvert_{\tau^*}$ at lots of different values of $\tau$ and find one that is suitably close to $\frac{1}{R}$ (i.e. I can numerically approximate a solution). Of course, I can only do this once I already have estimates of $\mu$ and $\sigma^2$ to instantiate $P(t)$, so let's start by fitting these to that bad data from the last section.

However, last time we only had one parameter to fit, $\lambda$, so we only needed one equation (which we got via the quantile function for the exponential distribution). This time, we have two, $\mu$ and $\sigma^2$, so we'll need _two_ equations. Luckily, if we use the whole sentence

> Overall, 99.7 percent of passengers waited less than 30 minutes and 91.2 percent of passengers waited less than 15 minutes.

We can get two equations involving the quantile function to solve! Let's try this below (I'll be doing this in linear algebra form to save myself some lines, so apologies if the math isn't super clear).

In [2]:
from bokeh.models import Range1d, LinearAxis

Fs = [15, 30]
log_Fs = np.array([np.log(x) for x in Fs])
ps = [0.912, 0.997]
a = [[1, np.sqrt(2)*(2*p - 1)] for p in ps]
a_inv = np.linalg.inv(a)

mu, sigma = a_inv @ log_Fs
dist = lognorm(s=sigma, scale=np.exp(mu))
max_val = dist.pdf(np.linspace(0, 20, 100)).max()

plot_kwargs = {
    "title": "Likelihood of Wait Times",
    "x_axis_label": "Time (minutes)",
    "y_axis_label": "Probability",
    "y_range": (0, 1.02*max_val)
}
p = plot_utils.plot_distributions({"PDF": dist}, x_max=20, plot_kwargs=plot_kwargs)

# plot the CDF as well on a second axis for clarity
cdf_x = np.linspace(0, 20, 100)
cdf_y = dist.cdf(cdf_x)
p.extra_y_ranges = {"cdf": Range1d(start=0, end=1)}
p.line(x=cdf_x, y=cdf_y, line_color=plot_utils.palette[1], legend_label="CDF", y_range_name="cdf")
p.add_layout(LinearAxis(y_range_name="cdf", axis_label="Cumulative Probability"), "right")

p.legend.title = "MLE Estimate: Mu={:0.2f}, Sigma={:0.2f}".format(mu, sigma)
p.legend.location = "center_right"
show(p)

Unsurprisingly, given the quality of our data, this ends up looking a lot like an exponential distribution. But in what seems like a shocking twist, it looks even _worse_ than the exponential from last time, indicating that we'll wait less than _5 minutes_ 80% of the time! Why would a seemingly better distribution behave this way?

Well, it turns out that one extra parameter makes a big difference in terms of how our model fits data. This added flexibility (which you might call a __degree of freedom__ if you felt like sounding technical) allows the model to fit this bad data even more tightly. Better models don't mean much when we don't have better data to feed them with (to borrow an overused phrase, garbage in-garbage out). It seems that if we're going to get serious about solving this problem for real answers, we're going to need to find better data. Data that
- is less obviously biased
- can be specifically targeted to days and times with representative traffic
- has many more observations

As it happens, the TSA released a few months of throughput data through the Freedom of Information Act back in October of 2019, which I managed to scrape at the time and save to csvs which I've included in this repository. Unfortunately, the links I used to scrape them don't seem to work anymore, and I'm having trouble finding the original data sources (which I probably should have saved the pdfs of, but alas...). In the next section, we'll play with this data, which is pretty interesting in its own right, and think about ways we can use it to help solve our problem.

For posterity, let's compute $\tau^*$ numerically as a function of $R$ and plot that to close the loop on this section (and our crappy data).

In [3]:
def f(R, grid_size=1000):
    '''
    Super inefficient grid-search based
    numerical approximation of ideal \tau
    value as a function of vector of Rs
    '''
    taus = []
    for r in R:
        target_derivative = 1./(r*60)
        derivative_estimate = np.inf
        tau = None
        x_min, x_max = 1, 100
        while not np.isclose(derivative_estimate, target_derivative, rtol=1e-3):
            grid = np.linspace(x_min, x_max, grid_size)
            d_tau = (x_max - x_min) / (grid_size)
            log_cdf = np.log(dist.cdf(grid))
            derivatives = np.diff(log_cdf) / d_tau

            argmin = np.argmin(np.abs(derivatives - target_derivative))
            derivative_estimate = derivatives[argmin]
            x_min, x_max = grid[argmin], grid[argmin+1]
            tau = ((grid[:-1] + grid[1:]) / 2)[argmin]
        taus.append(tau)
    return taus

plot_kwargs = {
    "title": "Cost Ratio vs. Ideal Arrival Time",
    "x_axis_label": "Cost Ratio (hours)",
    "y_axis_label": "Ideal Arrival Time (minutes)"
}
p = plot_utils.plot_functions(f, x_min=0.5, x_max=8, plot_kwargs=plot_kwargs)
show(p)

So, as we expected, the ideal arrival times as computed by this model are very low, especially at low values of $R$. Let's move on to greener pastures and better data, and see what sort of insights we can derive.